In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')

print("============================================")
print("/  MoteBattery is running.                 /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']
_DB_HOST  = config['kanjidb']['dbhost']
_DB_PORT  = config['kanjidb']['dbport']
_DB_NAME  = config['kanjidb']['dbname']
_DB_USER  = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']

_BATTERY_CRITICAL_VOLTAGE = float(config['analytics']['batterymin'])
_SLACK_BATTERYCHANNEL_NAME = config['analytics']['batterychannelname']
_SLACK_BATTERYCHANNEL_DBID = int(config['analytics']['batterychannelid'])

_USE_DROPBOX   = config['dropbox']['usedropbox']
_FALLBACK_IMAGE = config['dropbox']['fallbackimage']

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  logger(_LOG_INFO, "Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  logger(_LOG_ERROR, "You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  MoteBattery is running.                 /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
#_LOG_LEVEL = _LOG_DEBUG
_INTERVAL_MINUTES = 20

# Reissue Alerts on OPEN tickets every 15minutes
_TICKETAGE_REISSUE_THRESHOLD_SECONDS = 15 * 60

_PREFERRED_IMAGE =  "https://www.dropbox.com/s/8ckhly4i6a2umq9/battery-icon.jpg?raw=1"

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE

# If mote is not seen for 15minutes, generate a ticket
_MAX_MOTE_AGE_SECONDS = 15 * 60

#Ticket Type
MOTE_BATTERY_LOW = 10004
LORA_BATTERY_MOTE = 10002

#_BATTERY_CRITICAL_VOLTAGE = 3.6
logger(_LOG_INFO, "Critical battery voltage {}volts.".format(_BATTERY_CRITICAL_VOLTAGE))

now = datetime.now(pytz.utc)  #tz Aware
starttime = now - timedelta(hours=0, minutes=_INTERVAL_MINUTES)
logger(_LOG_DEBUG, "Current time is {}".format(now))
logger(_LOG_DEBUG, "Query timestamp will start at {}".format(starttime))

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

motequery = "SELECT * FROM kanji_node WHERE deploystate_id=10001 AND nodetype_id={}".format(LORA_BATTERY_MOTE)
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of motes {}".format(len(df.index)))
for ind in df.index:
    nodename = df['name'][ind]
    node_id = df['idnode'][ind]
    eventquery = "SELECT * FROM kanji_eventlog WHERE node_id={} AND timestamp > '{}' ORDER BY timestamp desc;".format(node_id, starttime)
    logger(_LOG_DEBUG, eventquery)
    df2 = pd.read_sql(eventquery, conn)
    samplesize = len(df2.index)
    logger(_LOG_DEBUG, "samplesize {}".format(samplesize))
    if samplesize!=0:
      voltageWma = 0.0
      divisor = 0  
      for ind2 in df2.index:
        #logger(_LOG_ERROR, df2['timestamp'][ind2])
        data = json.loads(df2['eventdata'][ind2].replace("'","\"")) 
        batteryvoltage = float(data['batteryvoltage'])
        voltageWma += batteryvoltage * (samplesize - ind2)
        divisor += (samplesize - ind2)
        logger(_LOG_DEBUG, "batteryvoltage  {} wma {}".format(batteryvoltage, voltageWma))
      voltageWma = voltageWma/divisor
      logger(_LOG_INFO, "{} battery={:3.1f}volts".format(nodename, voltageWma))
      if voltageWma < _BATTERY_CRITICAL_VOLTAGE:
        logger(_LOG_DEBUG, "Create ticket for mote {} battery low".format(nodename))
        location_id = df['location_id'][ind]
        locationquery = "SELECT location.idlocation, location.description, location.imageurl, location.slackalertchannel_id, \
                         customer.slacktoken, slackchannel.idslackchannel, slackchannel.channelname, slackchannel.channelid\
                         FROM kanji_location location \
                         JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
                         JOIN kanji_slackchannel slackchannel ON customer.slacksystemticketchannel_id=slackchannel.idslackchannel \
                         WHERE idlocation={}".format(location_id)
        
        df3 = pd.read_sql(locationquery, conn)
        locationid = df3["idlocation"][0]        
        
        logger(_LOG_DEBUG, "ImageUrl: {}".format(locationimageurl))        
        logger(_LOG_DEBUG, locationimageurl)
        locationdescription = df3["description"][0]            
        logger(_LOG_DEBUG, "locationdata")
        logger(_LOG_DEBUG, "locationQuery={}".format(locationquery))
        
        _SLACK_TOKEN = df3["slacktoken"][0]   
        
        logger(_LOG_INFO, "below threshold")
        description = "low battery voltage. ({:3.1f}volts)".format(voltageWma)
        mentions = ""
        #generate and Slack a new ticket ONLY if there is not a currently open ticket for this issue
        openTicket = kt.ticketExists(conn, node_id, MOTE_BATTERY_LOW, [kt._OPEN_STATUS, kt._WORKING_STATUS])
        if openTicket is None:
          ticketid = kt.openticket(conn, node_id, locationid, description, 2, 3, MOTE_BATTERY_LOW, _SLACK_BATTERYCHANNEL_DBID)
          ts = kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_BATTERYCHANNEL_NAME, ticketid, 0)
          kt.updateTicket(conn, ticketid, ts)  
          logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
        else:
          logger(_LOG_DEBUG, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))
      else:
        logger(_LOG_INFO, "--Ok")
        
logger(_LOG_INFO, "MoteBattery Done!") 

Critical battery voltage 3.65volts.
agMote-20009 battery=3.8volts
--Ok
agMote-20002 battery=4.0volts
--Ok
agMote-20010 battery=3.8volts
--Ok
agMote-20003 battery=3.9volts
--Ok
MoteBattery Done!
